In [2]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
# 2. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import numpy as np
from PIL import Image,ImageEnhance
import cv2

from google.colab.patches import cv2_imshow

In [31]:
def enhancer(im,a):
    #im = Image.open("drive/My Drive/Results 3/result0244 enhanced-a=0.6,b=0.5,g=-0.05,d=0.05 .JPG")
    enhancer = ImageEnhance.Brightness(im)
    enhanced_im = enhancer.enhance(a)
    enhanced_im.save("drive/My Drive/enhance/enhanced_sample.JPG")
    return 

In [15]:
def find_nearest_above(my_array, target):
    diff = my_array - target
    mask = np.ma.less_equal(diff, -1)
    # We need to mask the negative differences
    # since we are looking for values above
    if np.all(mask):
        c = np.abs(diff).argmin()
        return c # returns min index of the nearest if target is greater than any value
    masked_diff = np.ma.masked_array(diff, mask)
    return masked_diff.argmin()

In [16]:
def hist_match(original, specified):

    oldshape = original.shape
    original = original.ravel()
    specified = specified.ravel()

    # get the set of unique pixel values and their corresponding indices and counts
    s_values, bin_idx, s_counts = np.unique(original, return_inverse=True,return_counts=True)
    t_values, t_counts = np.unique(specified, return_counts=True)

    # Calculate s_k for original image
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    
    # Calculate s_k for specified image
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # Round the values
    sour = np.around(s_quantiles*255)
    temp = np.around(t_quantiles*255)
    
    # Map the rounded values
    b=[]
    for data in sour[:]:
        b.append(find_nearest_above(temp,data))
    b= np.array(b,dtype='uint8')

    return b[bin_idx].reshape(oldshape)

In [49]:
def histmatch(path):
    Img1 = cv2.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
    Img2 = cv2.imread(path)
    l = Img1.shape[0]
    b = Img1.shape[1]  
    sumb = np.sum(Img2[:,:,0])
    x = sumb/(l*b)  
    final1 = hist_match(Img2, Img1)
    cv2.imwrite("drive/My Drive/enhance/enhanced_sample.JPG",final1)
    im2 = Image.open("drive/My Drive/enhance/enhanced_sample.JPG")
    enhancer(im2,1.4)

    final = cv2.imread("drive/My Drive/enhance/enhanced_sample.JPG")

    for i in range(l):
        print(i)
        for j in range(b):
            if Img2[i][j][0] > (0.75*x):  
                  final[i][j] = Img2[i][j]
    cv2.imwrite("drive/My Drive/Results_ycc_coarse/0244histspec* 1.4.JPG", final)         



In [ ]:
histmatch("drive/My Drive/Images_for_Adarsh/DSC_0244.JPG")

In [1]:
def brighten_YCbCr(path,alpha,beta):
    Img1 = cv2.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
    Img2 = cv2.imread(path)
    ycc1 = cv2.cvtColor(Img1,cv2.COLOR_BGR2YCR_CB)
    ycc2 = cv2.cvtColor(Img2,cv2.COLOR_BGR2YCR_CB)
    sumb = np.sum(Img2[:,:,0])
    diffv = ycc1 - ycc2
    diffv = np.array(diffv)
    sumy = np.sum(np.square(diffv[:,:,0]))
    #sumy = np.sum(diffv[:,:,0])
    l = ycc1.shape[0]
    b = ycc1.shape[1]
    count = l*b
    y = np.sqrt(sumy/count)
    #y = sumy/count
    x = sumb/count
    #p = np.minimum(255, np.maximum((alpha*hsv2[:,:,2]) + (beta*a), 0))
    p = (alpha*ycc2[:,:,0]) + (beta*y)
    ycc2[:,:,0] = p

    final1 = cv2.cvtColor(ycc2,cv2.COLOR_YCR_CB2BGR)
    #cv2.imwrite("drive/My Drive/enhance/enhanced_sample.JPG",final1)
    #im2 = Image.open("drive/My Drive/enhance/enhanced_sample.JPG")
    #enhancer(im2,1.5)
    #final1 = cv2.imread("drive/My Drive/enhance/enhanced_sample.JPG")
    final = hist_match(final1, Img1)
    for i in range(l):
        print(i)
        for j in range(b):
            if Img2[i][j][0] > (0.75*x):  
                  final[i][j] = Img2[i][j]


    cv2.imwrite("drive/My Drive/Results_ycc_coarse/6181_ycc histspec1-a=1.5,b=1.5.JPG", final)         




In [ ]:
brighten_YCbCr("drive/My Drive/Images_for_Adarsh/DSC_6181.JPG",1.5,1.5)

In [ ]:
# def brighten_YCbCrreplace(path):
#     Img1 = cv2.imread("drive/My Drive/Images_for_Adarsh/DSC_6139.JPG")
#     Img2 = cv2.imread(path)
#     ycc1 = cv2.cvtColor(Img1,cv2.COLOR_BGR2YCR_CB)
#     ycc2 = cv2.cvtColor(Img2,cv2.COLOR_BGR2YCR_CB)
#     ycc2[:,:,0] = ycc1[:,:,0]
#     sumb = np.sum(Img2[:,:,0])
#     l = ycc1.shape[0]
#     b = ycc1.shape[1]
#     count = l*b
#     x = sumb/count
#     final = cv2.cvtColor(ycc2,cv2.COLOR_YCR_CB2BGR)

#     for i in range(l):
#         print(i)
#         for j in range(b):
#             if Img2[i][j][0] > (0.75*x):  
#                   final[i][j] = Img2[i][j]


#     cv2.imwrite("drive/My Drive/Results_ycc_coarse/6181_ycc-replace.JPG", final)       

In [ ]:
#brighten_YCbCrreplace("drive/My Drive/Images_for_Adarsh/DSC_6181.JPG")

In [ ]:
a = hist_match(original, specified)